In [107]:
import torch
import matplotlib.pyplot as plt
import torchvision
import cv2
import glob
import os

In [21]:
def calculate_distance(point, point2):
    px, py = point
    x, y = point2
    return ((px - x) ** 2 + (py - y) ** 2) ** 0.5

In [46]:
def phone_use(people, phones):
    phone_use = {}
    for id, phone in phones.items():
        distance = 1000000
        for p_id, person in people.items():
            new_distance = calculate_distance((person["bbox"][0]+person["bbox"][2]/2, person["bbox"][1]+person["bbox"][3]/2), (phone["bbox"][0]+phone["bbox"][2]/2, phone["bbox"][1]+phone["bbox"][3]/2))
            if new_distance < distance:
                distance = new_distance
                phone_use[id] = p_id

    return phone_use

In [28]:
def extract_annotation(results):
    annotations = []
    for result in results:
        bbox = result[0], result[1], result[2], result[3]
        certainty = result[4]
        category = result[5]
        area = result[0]*result[1] 
        annotation = {"area": area, "bbox": bbox, "certainty": certainty, "category_id": category}
        annotations.append(annotation)
    return annotations

In [76]:
def draw_label(image, label, bbox, color):
    x, y, x2, y2 = bbox
    x, y, x2, y2 = int(x), int(y), int(x2), int(y2)
    cv2.rectangle(image, (x, y), (x2, y2), color, 2)
    (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(image, (x, y - text_height - baseline), (x + text_width, y), color, thickness=cv2.FILLED)
    cv2.putText(image, label, (x, y - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    return image

In [109]:
PASSENGER = (231, 48, 122)
PASSENGER_USE = (4, 255, 1)
HAND = (158, 124, 101)
DRIVER = (126, 45, 34)
DRIVER_USE = (0, 3, 252)

def run_yolo(model,image_path):
    """ 
    ids = {
        CIH__id or hand: 0,
        Cell_Mounted: 4,
        face_id: 1,
    }
    """
    image_result = model(image_path)
    image = cv2.imread(image_path)
    #print(image_result.pred[0])

    _, width, _ = image.shape

    people = {}
    phones = {}
    hands = {}

    annotations = extract_annotation(image_result.pred[0])
    num_faces = 0

    for annotation in annotations:
        if annotation['category_id'] == 1:
            people[annotation["area"]] = annotation
            num_faces += 1
        elif annotation["category_id"] == 4 or annotation["category_id"] == 0:
            annotation['category_id'] = "Phone"
            phones[annotation["area"]] = annotation
        elif annotation["category_id"] == 2:
            annotation['category_id'] = "Hand"
            hands[annotation["area"]] = annotation

    for annotation in annotations:
        if annotation["category_id"] == 1:
            if annotation["area"] < 5000 and num_faces > 1:
                people[annotation["area"]]['category_id'] = "Passenger"
            elif annotation["bbox"][0] < width / 2 and num_faces > 1:
                people[annotation["area"]]['category_id'] = "Passenger"
            else:
                people[annotation["area"]]['category_id'] = "Driver"
                driver = annotation["area"]


    phone_use_list = phone_use(people, phones)
    for phone, person in phone_use_list.items():
        if people[person]["category_id"] == "Driver":
            phones[phone]["category_id"] = "Driver Use"
        else:
            phones[phone]["category_id"] = "Passenger Use"

        
    new_annotations = list(people.values()) + list(phones.values()) + list(hands.values())

    driver_use = False
    for ann in new_annotations:
        if ann["category_id"] == "Driver":
            image = draw_label(image, ann["category_id"], ann["bbox"], DRIVER)
        elif ann["category_id"] == "Passenger":
            image = draw_label(image, ann["category_id"], ann["bbox"], PASSENGER)
        elif ann["category_id"] == "Driver Use":
            image = draw_label(image, ann["category_id"], ann["bbox"], DRIVER_USE)
            driver_use = True
        elif ann["category_id"] == "Passenger Use":
            image = draw_label(image, ann["category_id"], ann["bbox"], PASSENGER_USE)
        elif ann["category_id"] == "Hand":
            image = draw_label(image, ann["category_id"], ann["bbox"], HAND)
        

    #image_result.save()
    
    return image, driver_use

In [7]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath


# To run the model You just need to clone yolov5 repository from github 
# and add new_yolo_2.pt into yolov5\models\ directory
# https://github.com/ultralytics/yolov5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.hub.load(r"yolov5", 'custom', path=r"yolov5\models\new_yolo_2.pt", source='local') 

#model.conf = 0.50  # confidence threshold (0-1)
#model.iou = 0.45  # NMS IoU threshold (0-1)  

model = model.to(device)

YOLOv5  v7.0-330-gb20fa802 Python-3.11.9 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 212 layers, 20869098 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [103]:
result = run_yolo(model, 'PREPROCESSED_IMAGES/1/Image852.jpg')

Saved 1 image to runs\detect\exp14


tensor([[7.72955e+02, 3.74971e+02, 8.87851e+02, 5.29964e+02, 9.69052e-01, 1.00000e+00],
        [2.63338e+02, 3.40097e+02, 3.71212e+02, 4.74920e+02, 9.68198e-01, 1.00000e+00],
        [6.29446e+02, 5.17787e+02, 7.42793e+02, 6.42067e+02, 8.59240e-01, 2.00000e+00],
        [6.21837e+02, 4.32987e+02, 7.50511e+02, 6.05201e+02, 8.53580e-01, 0.00000e+00],
        [1.11253e+03, 4.79347e+02, 1.20096e+03, 5.31686e+02, 8.25810e-01, 2.00000e+00],
        [2.45718e+02, 4.41311e+02, 3.01816e+02, 5.39711e+02, 7.37672e-01, 2.00000e+00]])


In [104]:
cv2.imwrite("test.jpg", result)

True

In [110]:
# Class Names: ["Cell Phone", "Face", "Hand", "OOD", "Cell Phone Attached"]

results = {}
# this is sort of true the ids dont really mean this
ids = {
    "CIH__id": 0,
    "Cell": 4,
    "face_id": 1,
    "hand": 2
}


status = ["0","1"]
labels,inputs = [], [] # not to sure
folder_destination = "PROCESSED/"
os.makedirs(folder_destination, exist_ok=True)
for i,s in enumerate(status):
    for index,image_path in enumerate(glob.glob("BLURRED_IMAGES\\"+s+"\\*")):
        img_name = os.path.basename(image_path)
        processed_image, driver_use = run_yolo(model, image_path)
        labels.append(driver_use)
        inputs.append(processed_image)
        if driver_use:
            cv2.imwrite(folder_destination+"1/"+img_name, processed_image)
        else:
            cv2.imwrite(folder_destination+"0/"+img_name, processed_image)
        


In [ ]:
def run_yolo_hand_Extraction(model,image_path,ids):
    """
    ids = {
        CIH__id: 0,
        Cell: 4,
        face_id: 1,
    }
    """
    
    results = model(image_path)

    """
    PART 3: FILL THIS AREA
    """
        
    return 0


status = ["1","0"]
labels,inputs = [], []

ids = {
    "CIH__id": 0,
    "Cell": 4,
    "face_id": 1,
}

for i,s in enumerate(status):
    for index,image_path in enumerate(glob.glob("BLURRED_IMAGES\\"+s+"\\*")):
        """
        PART 3: FILL THIS AREA
        """
